In [75]:
import glob
import pandas as pd
import json
import os

find_path = "/Users/joon/Desktop/차량 내 탑승자 상황 인식 영상 데이터/데이터/Validation/라벨링데이터/abnormal_230303_add/VL1"
file_list = glob.glob(os.path.join(find_path, "*/*/label/*.json"))

len(file_list)

50496

In [60]:
# 모든 데이터에 occupant는 한명
check = False

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())
    if len(js['occupant_info']) != 1:
        check = True
        break

print(check)

False


In [62]:
# 모든 이미지에 occupant는 한명
check = False

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())

    if len(js['occupant_info']) != 1:
        check = True
    
    scene = js['scene']['data']
    for img in scene:
        if len(img['occupant']) != 1:
            check = True
    if check:
        break

print(check)

False


# 데이터 변환

In [69]:
data = []

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())

    video_id = js['metadata']['video_id']
    date = js['metadata']['date']
    scene_id = js['scene_info']['scene_id']
    category_id = js['scene_info']['category_id']
    category_name = js['scene_info']['category_name']
    occupant_id = js['occupant_info'][0]['occupant_id']
    occupant_sex = js['occupant_info'][0]['occupant_sex']
    occupant_age = js['occupant_info'][0]['occupant_age']
    occupant_position = js['occupant_info'][0]['occupant_posotion'] # posotion이라고 잘못되어있음

    scene = js['scene']['data']
    for img in scene:
        data_length = len(scene)
        img_id = img['img_name']
        # occupant_id = img['occupant'][0]['occupant_id'] # 이미 있음
        b_bbox1, b_bbox2, b_bbox3, b_bbox4 = img['occupant'][0]['body_b_box']
        f_bbox1, f_bbox2, f_bbox3, f_bbox4 = img['occupant'][0]['face_b_box']
        action = img['occupant'][0]['action']
        emotion = img['occupant'][0]['emotion']

        data.append([video_id, date, scene_id, category_id, category_name, occupant_id, occupant_sex, occupant_age, occupant_position, data_length, img_id, b_bbox1, b_bbox2, b_bbox3, b_bbox4, f_bbox1, f_bbox2, f_bbox3, f_bbox4, action, emotion])

In [70]:
df = pd.DataFrame(data=data, columns=['video_id', 'date', 'scene_id', 'category_id', 'category_name', 'occupant_id', 'occupant_sex', 'occupant_age', 'occupant_position', 'data_length', 'img_id', 'b_bbox1', 'b_bbox2', 'b_bbox3', 'b_bbox4', 'f_bbox1', 'f_bbox2', 'f_bbox3', 'f_bbox4', 'action', 'emotion'])

In [73]:
df

,video_id,date,scene_id,category_id,category_name,occupant_id,occupant_sex,occupant_age,occupant_position,data_length,...,b_bbox1,b_bbox2,b_bbox3,b_bbox4,f_bbox1,f_bbox2,f_bbox3,f_bbox4,action,emotion
0,SGA5100028,20211111,SGA5100028S0074,A004,통화,occupant1,M,30대,front,5,...,449.63999,82.600000,830.76001,636.200000,700.83999,97.000000,261.16001,271.400000,운전하다,중립
1,SGA5100028,20211111,SGA5100028S0074,A004,통화,occupant1,M,30대,front,5,...,416.03999,82.600000,867.56001,639.400000,698.60000,89.000000,263.40000,276.200000,무언가를쥐다,중립
2,SGA5100028,20211111,SGA5100028S0074,A004,통화,occupant1,M,30대,front,5,...,409.00000,85.800000,871.40000,628.200000,589.80000,98.600000,295.40000,306.600000,고개를돌리다,중립
3,SGA5100028,20211111,SGA5100028S0074,A004,통화,occupant1,M,30대,front,5,...,415.40000,87.400000,866.60000,628.200000,660.20000,81.000000,300.20000,297.000000,무언가를쥐다,중립
4,SGA5100028,20211111,SGA5100028S0074,A004,통화,occupant1,M,30대,front,5,...,425.00000,92.200000,847.40000,628.200000,690.60000,92.200000,268.20000,284.200000,운전하다,중립
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252472,SGA5100053,20211111,SGA5100053S0063,A004,통화,occupant1,M,30대,front,5,...,396.20000,39.427022,885.80000,682.572978,676.20000,53.815644,270.43999,311.400000,운전하다,중립
252473,SGA5100053,20211111,SGA5100053S0063,A004,통화,occupant1,M,30대,front,5,...,372.20000,49.044089,858.60000,672.955911,676.20000,68.217067,274.60000,306.571272,핸드폰귀에대기,중립
252474,SGA5100053,20211111,SGA5100053S0063,A004,통화,occupant1,M,30대,front,5,...,354.60000,29.804267,927.40000,692.195733,674.60000,63.398578,283.23999,321.000000,핸드폰귀에대기,중립
252475,SGA5100053,20211111,SGA5100053S0063,A004,통화,occupant1,M,30대,front,5,...,346.60000,18.607111,935.40000,703.392889,671.40000,49.007111,278.43999,330.576961,핸드폰귀에대기,중립


In [72]:
df.describe()

,data_length,b_bbox1,b_bbox2,b_bbox3,b_bbox4,f_bbox1,f_bbox2,f_bbox3,f_bbox4
count,252477.000000,252477.000000,252477.000000,252477.000000,252477.000000,252477.000000,252477.000000,252477.000000,252477.000000
mean,4.999952,488.724049,116.945151,730.811104,598.535988,682.900218,135.950699,261.621394,295.670476
std,0.006894,100.261705,64.350843,134.367988,65.663735,119.023849,62.557658,60.642100,54.061363
min,4.000000,-423.800000,-321.400000,73.287500,33.128000,-243.800000,-298.200000,73.287500,33.128000
25%,5.000000,448.074982,68.214222,633.600000,565.450928,655.400000,85.125000,212.200000,258.005263
50%,5.000000,498.600000,110.145464,741.800000,600.379759,713.000000,137.671112,268.200000,297.000000
75%,5.000000,543.400000,151.725000,811.560010,649.000000,753.000000,169.000000,295.400000,327.013065
max,5.000000,1650.600000,602.600000,1287.400000,745.000000,1116.839990,561.310490,1261.800000,698.599999


In [74]:
df.to_csv('./validation.csv', index=False)